In [1]:
import snscrape.modules.twitter as sntwitter
import snscrape
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
# Define the Twitter username
username = "Starbucks"

# Define the date range
start_date = dt.date(2008, 1, 1)
end_date = dt.date(2022, 12, 1)

# Define the exclude filter for replies
exclude_filter = "exclude:replies"

# Create a list to store the tweets
tweets = []

# Use snscrape to search for tweets from the user within the date range
for tweet in sntwitter.TwitterSearchScraper("from:" + username + " since:" + str(start_date) + " until:" + str(end_date) + " " + exclude_filter).get_items():
    #ignore tweets with no media
    if tweet.media:
        tweets.append(tweet)


    

In [68]:
id = ['s'+ str(tweet.id) for tweet in tweets]



# Create a list of tweet content
content = [tweet.content for tweet in tweets]

# Create a list of tweet media urls
media_urls = [tweet.media for tweet in tweets]


# Create a list of tweet hashtags
hashtags = [tweet.hashtags for tweet in tweets]

# Create a list of likes

likes = [tweet.likeCount for tweet in tweets]

# Create a list of retweets
retweets = [tweet.retweetCount for tweet in tweets]

# Create a list of replies
replies = [tweet.replyCount for tweet in tweets]

# Create a list of tweet URLs
urls = [tweet.url for tweet in tweets]

#create a list of tweet dates
dates = [tweet.date for tweet in tweets]

C:\Users\vinay\AppData\Local\Temp\ipykernel_18304\3137642976.py:6: FutureWarning: content is deprecated, use rawContent instead
  content = [tweet.content for tweet in tweets]


In [3]:
tweets[0]

Tweet(url='https://twitter.com/Starbucks/status/1593741479335587840', date=datetime.datetime(2022, 11, 18, 23, 2, 33, tzinfo=datetime.timezone.utc), rawContent='When you open a Starbucks Reserve® store in an NYC landmark, you celebrate with an NYC icon. @aliciakeys joined us for the lighting ceremony—the first of many unforgettable experiences here at the @EmpireStateBldg. Hope to see you soon. ✨ #StarbucksReserve #AliciaSantaBaby https://t.co/QnaIZCLHpq', renderedContent='When you open a Starbucks Reserve® store in an NYC landmark, you celebrate with an NYC icon. @aliciakeys joined us for the lighting ceremony—the first of many unforgettable experiences here at the @EmpireStateBldg. Hope to see you soon. ✨ #StarbucksReserve #AliciaSantaBaby https://t.co/QnaIZCLHpq', id=1593741479335587840, user=User(username='Starbucks', id=30973, displayname='Starbucks Coffee', rawDescription='Make today a good day. 💚', renderedDescription='Make today a good day. 💚', descriptionLinks=None, verified=T

In [69]:
media_type = []
media_url = []

for tweet in tweets:

    m_t = []
    m_u = []
    for medium in tweet.media:
    
        if isinstance(medium, snscrape.modules.twitter.Photo):
            m_t.append("photo")
            m_u.append(medium.fullUrl)
        elif isinstance(medium, snscrape.modules.twitter.Video):
            m_t.append("video")
            m_u.append(medium.variants[0].url)
        elif isinstance(medium, snscrape.modules.twitter.Gif):
            m_t.append("gif")
            m_u.append(medium.variants[0].url)
    media_type.append(m_t)
    media_url.append(m_u)


177
177
177
177
177
177


In [70]:
#create a dataframe with the data

df = pd.DataFrame({'id': id,'date':dates, 'content': content,'url':urls ,'likes':likes,'retweets':retweets,'reply_count': replies ,'hashtags': hashtags, 'media_type': media_type, 'media_url': media_url})


#create a new row if there are multiple elements in the media_url column and media_type column in such a way that first element of media_url and media_type is in the same row as the rest of the elements

df = df.explode(list('media_url,media_type'.split(',')))

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1625 entries, 0 to 1509
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   id           1625 non-null   object             
 1   date         1625 non-null   datetime64[ns, UTC]
 2   content      1625 non-null   object             
 3   url          1625 non-null   object             
 4   likes        1625 non-null   int64              
 5   retweets     1625 non-null   int64              
 6   reply_count  1625 non-null   int64              
 7   hashtags     1118 non-null   object             
 8   media_type   1625 non-null   object             
 9   media_url    1625 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(3), object(6)
memory usage: 139.6+ KB


In [56]:
df.to_csv('starbucks.csv', index=False)

In [71]:
import requests
import os 


In [72]:
#create a folder to store the images and videos if it doesn't exist
folder_name = username + '_media'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    

()

In [60]:
#download the images and videos and save them in the folder

for index, row in df.iterrows():
    if row['media_type'] == 'photo':
        r = requests.get(row['media_url'])
        with open(folder_name + '/' + row['id'] + '.jpg', 'wb') as f:
            f.write(r.content)    
    elif row['media_type'] == 'video':
        
        r = requests.get(row['media_url'])
        with open(folder_name + '/' + row['id'] + '.mp4', 'wb') as f:
            f.write(r.content)

            
    elif row['media_type'] == 'gif':
        r = requests.get(row['media_url'])
        with open(folder_name + '/' + row['id'] + '.gif', 'wb') as f:
            f.write(r.content)

In [64]:
#create a column that contains the path to the media file give jpg for photos and mp4 for videos and gif for gifs

df['media_path'] = df.apply(lambda row: folder_name + '/' + row['id'] + '.jpg' if row['media_type'] == 'photo' else folder_name + '/' + row['id'] + '.mp4' if row['media_type'] == 'video' else folder_name + '/' + row['id'] + '.gif' if row['media_type'] == 'gif' else None, axis=1)

In [65]:

df.to_csv('starbucks.csv', index=False)